In [1]:
import pandas as pd
import numpy as np
from ast import literal_eval
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from model_trainer.trainers.tabnet_trainer import TabNetTrainer
from fastai.tabular.all import *

In [2]:
df = pd.read_csv('../../../data/dataset_initial.csv')
df['Weather_Condition_Arr'] = df['Weather_Condition_Arr'].apply(lambda x: literal_eval(x) if str(x)!='nan' else x)
X, y = df[df.columns.drop('Severity')], df['Severity']

In [3]:
X_train_val_test, Y_train_val_test = X.copy(),y.copy()
X_train_val,X_val,Y_train_val,Y_val = train_test_split(X_train_val_test, Y_train_val_test,train_size=0.8,stratify=Y_train_val_test)

In [4]:
from hyperopt import STATUS_OK,hp,tpe,Trials,fmin
from hyperopt.pyll import scope

tabnet_large_space={
    "lookahead": hp.choice("lookahead",[False,True]),
    "optimizer": hp.choice('optimizer',[
        {
            "opttype":"Adam",
             "wd":hp.loguniform('wdadam', np.log(0.0001), np.log(0.3))
        },
        {
            "opttype":"SGD",
            "wd":hp.loguniform('wdsgd', np.log(0.0001), np.log(0.3))
        },
        {
            "opttype":"RAdam",
            "wd":hp.loguniform('wdradam', np.log(0.0001), np.log(0.3))
        }
    ]),
    "n":scope.int(hp.choice("n",[8,64,128])),
    "n_steps":scope.int(hp.quniform("n_steps",3,10,1)),
    "gamma":hp.uniform("gamma",1,2),
    "momentum":hp.uniform("momentum",0,1),
    "lr":hp.choice("lr",[0.005,0.01,0.02,0.025]),
    "batch_size":hp.quniform("batch_size",12,17,1),
    "virtual_batch_size":hp.quniform("virtual_batch_size",8,11,1)
    }

In [5]:
def get_optimizer(opttype,opt_params,lookahead):
    OPT_DICT = {
        "Adam":Adam,
        "RAdam":RAdam,
        "SGD":SGD
    }
    opt_constructor = OPT_DICT[opttype]
    if lookahead:
        partial_opt = lambda spliter,lr: Lookahead(opt_constructor(spliter,lr,**opt_params))
        optimizer = partial_opt
    else:
        optimizer = partial(opt_constructor,**opt_params)
    return optimizer

def process_params(params):
    params['batch_size'] = int(np.power(2,params['batch_size']))
    params['virtual_batch_size'] = int(np.power(2,params['virtual_batch_size']))

    opt_params = params.pop('optimizer')
    opttype = opt_params.pop('opttype')
    lookahead = params.pop('lookahead')
    optimizer = get_optimizer(opttype,opt_params,lookahead)
    
    params['optimizer'] = optimizer
    n=params.pop('n')
    params['n_d']=n
    params['n_a']=n
    return params
    


Dumping trial object after each evaluation to preserve the state.

In [6]:
import os.path
from os import path


def tabnet_fn(params):
    params = process_params(params)
    print(params)
    tabnet_trainer = TabNetTrainer(X_train_val,Y_train_val,vtype="hold-out",split=0.8,epochs=50)
    return -tabnet_trainer.train_and_validate(params)
    
    
if path.exists("tabnet-trials.p"):
    trials = pickle.load(open("tybnet-trials.p"))
else:
    trials = Trials()   

best_hyperparams = fmin(fn = tabnet_fn,
                        space = tabnet_large_space,
                        algo = tpe.suggest,
                        max_evals = 1,
                        trials = trials)
   
pickle.dump(trials, open("tabnet-trials.p", "wb"))
best_hyperparam

{'batch_size': 4096, 'gamma': 1.5950412156732081, 'lr': 0.01, 'momentum': 0.352546163081788, 'n_steps': 7, 'virtual_batch_size': 512, 'optimizer': functools.partial(<function Adam at 0x7f932d13f2e0>, wd=0.002330993167666582), 'n_d': 8, 'n_a': 8}
  0%|                                                                                               | 0/1 [00:00<?, ?trial/s, best loss=?]

epoch,train_loss,valid_loss,matthews_corrcoef,time


  0%|                                                                                               | 0/1 [01:12<?, ?trial/s, best loss=?]


KeyboardInterrupt: 